In [72]:
import pandas as pd
import requests
import re
from IPython.display import display, HTML
from bs4 import BeautifulSoup
import io

In [73]:
def displaywithHeader(df, title) :
    display(HTML(f'<h3>{title}</h3>'))
    display(df.head(3))

### Get Family Group Table Information from the Improvement Service's Website

In [74]:
# Get Page HTML
html = requests.get('https://www.improvementservice.org.uk/benchmarking/how-do-we-compare-councils').content
html = BeautifulSoup(html, "lxml")
tables = html.findAll("table")
tables_list = []
for table in tables :
    table_str = str(table)
    table_str = table_str.replace('<br/>',';')
    tables_list.append(io.StringIO(table_str))
    
FamilyGroups_CSWH = pd.read_html(tables_list[0])[0]
FamilyGroups_ECLEDC = pd.read_html(tables_list[1])[0]

displaywithHeader(FamilyGroups_CSWH,'FamilyGroups_CSWH')
displaywithHeader(FamilyGroups_ECLEDC,'FamilyGroups_ECLEDC')

,Family Group 1,Family Group 2,Family Group 3,Family Group 4
0,East Renfrewshire;East Dunbartonshire;Aberdeen...,Moray;Stirling;East Lothian;Angus;Scottish Bor...,Falkirk;Dumfries & Galloway;Fife;South Ayrshir...,Eilean Siar;Dundee City;East Ayrshire;North Ay...


,Family Group 1,Family Group 2,Family Group 3,Family Group 4
0,Eilean Siar;Argyll & Bute;Shetland Islands;Hig...,Perth & Kinross;Stirling;Moray;South Ayrshire;...,Angus;Clackmannanshire;Midlothian;South Lanark...,North Lanarkshire;Falkirk;East Dunbartonshire;...


### Unpivot the Data

In [75]:
FamilyGroups_ECLEDC = FamilyGroups_ECLEDC.assign(id= 1)
FamilyGroups_ECLEDC = pd.melt(FamilyGroups_ECLEDC, id_vars=['id'], value_vars=['Family Group 1', 'Family Group 2', 'Family Group 3', 'Family Group 4'],var_name='Family_Group', value_name='Local_Authority')
FamilyGroups_ECLEDC = FamilyGroups_ECLEDC.drop(columns = ['id'])

FamilyGroups_CSWH = FamilyGroups_CSWH.assign(id= 1)
FamilyGroups_CSWH = pd.melt(FamilyGroups_CSWH, id_vars=['id'], value_vars=['Family Group 1', 'Family Group 2', 'Family Group 3', 'Family Group 4'],var_name='Family_Group', value_name='Local_Authority')
FamilyGroups_CSWH = FamilyGroups_CSWH.drop(columns = ['id'])

displaywithHeader(FamilyGroups_CSWH,'FamilyGroups_CSWH')
displaywithHeader(FamilyGroups_ECLEDC,'FamilyGroups_ECLEDC')

,Family_Group,Local_Authority
0,Family Group 1,East Renfrewshire;East Dunbartonshire;Aberdeen...
1,Family Group 2,Moray;Stirling;East Lothian;Angus;Scottish Bor...
2,Family Group 3,Falkirk;Dumfries & Galloway;Fife;South Ayrshir...


,Family_Group,Local_Authority
0,Family Group 1,Eilean Siar;Argyll & Bute;Shetland Islands;Hig...
1,Family Group 2,Perth & Kinross;Stirling;Moray;South Ayrshire;...
2,Family Group 3,Angus;Clackmannanshire;Midlothian;South Lanark...


### Split Into Rows

In [76]:
FamilyGroups_CSWH['Local_Authority'] = FamilyGroups_CSWH['Local_Authority'].str.split(';')
FamilyGroups_CSWH = FamilyGroups_CSWH.explode('Local_Authority')

FamilyGroups_ECLEDC['Local_Authority'] = FamilyGroups_ECLEDC['Local_Authority'].str.split(';')
FamilyGroups_ECLEDC = FamilyGroups_ECLEDC.explode('Local_Authority')

displaywithHeader(FamilyGroups_CSWH,'FamilyGroups_CSWH')
displaywithHeader(FamilyGroups_ECLEDC,'FamilyGroups_ECLEDC')

,Family_Group,Local_Authority
0,Family Group 1,East Renfrewshire
0,Family Group 1,East Dunbartonshire
0,Family Group 1,Aberdeenshire


,Family_Group,Local_Authority
0,Family Group 1,Eilean Siar
0,Family Group 1,Argyll & Bute
0,Family Group 1,Shetland Islands


### Add Type and Append

In [77]:
FamilyGroups_CSWH['Type'] = 'Children, Social Work and Housing indicators'
FamilyGroups_ECLEDC['Type'] = 'Environmental, Culture & Leisure, Economic Development, Corporate and Property indicators'

Family_Groups = pd.concat([FamilyGroups_ECLEDC, FamilyGroups_CSWH])

#There is a mismatch between the naming of Edinburgh on the Improvement Services Family Groupings web page and the naming in the raw data file. Replace the text here to allow merges with the raw data file later.
Family_Groups['Local_Authority'] = Family_Groups['Local_Authority'].str.replace('Edinburgh, City of','Edinburgh City')

displaywithHeader(Family_Groups,'Family_Groups')

,Family_Group,Local_Authority,Type
0,Family Group 1,Eilean Siar,"Environmental, Culture & Leisure, Economic Dev..."
0,Family Group 1,Argyll & Bute,"Environmental, Culture & Leisure, Economic Dev..."
0,Family Group 1,Shetland Islands,"Environmental, Culture & Leisure, Economic Dev..."


In [78]:
Family_Groups.to_csv("Data Files//Family Groups.csv", index = False, encoding='utf-8-sig')